In [1]:
import pydicom
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
import cv2  
import operator 
from collections import Counter
import open3d as o3d
import pyvista as pv
from pyvista import examples
import pandas as pd
import gmsh
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def multiplo5(n):
    if n%5 == 0:
        return True
    else:
        return False

In [2]:
class imagen:
    def __init__ (self, slice, time, img, info, dir):
        self.slice = slice
        self.time = time
        self.img = img
        self.info = info
        self.dir = dir 

In [4]:
SeriesInstance = pydicom.uid.generate_uid()

def addDCM(DCMorigen,PathFull,ImgName,sizex,sizey,mapa):

        arr=np.zeros((sizex,sizey),dtype=np.int16)
        for i in range(sizex):
            for j in range (sizey):
                arr[i,j]=mapa[i,j]
        
        ds=DCMorigen
        #arr=ds.pixel_array
        #arr[arr<20]=0
        #arr=mapa
        #arr=np.array(mapa)
        #print("Array del mapa 70,159:",arrMapa[70,159])
        #print(arrMapa)
        plt.imshow(arr,cmap=plt.cm.bone)
        plt.show()
        
        modificationTime=time.strftime("%H%M%S")
        pathFinal=PathFull+"/"+ImgName+".dcm"#path completo con que se va a guardar cada imagen dicom final
        #np_frame1=listofmaps[k]
        #ds.Rows=geometry[6] #np_frame1.shape[0]
        #ds.Columns=geometry[7] #np_frame1.shape[1]
        ds[0x0020,0x0011].value=np.random.randint(200,500)
        ds.PhotometricInterpretation="MONOCHROME2"
        #ds.SamplesPerPixel=1
        #ds.BitsStored=16
        #ds.BitsAllocated=16
        #ds.HighBit=15
        #ds.PixelRepresentation=1
        #ds.PixelData=arrMapa.tobytes()
        ds.SeriesDescription= 'prueba21' #'MAP'+str(k)
        ds.SeriesInstanceUID= SeriesInstance
        ds[0x008,0x0031].value=modificationTime
        ds[0x008,0x0018].value=ds[0x008,0x0018].value+str(1)
        ds[0x008,0x0008].value='SECONDARY'
        ds.PixelData=arr.tobytes()
        ds.save_as(pathFinal)
        print("path final:",pathFinal)

In [5]:
def position_ROI(Dcm_im_ROI):
    '''
    Give the general position of the ROI used to be copy on every image
    '''
    Dcm_im8 = (Dcm_im_ROI).astype(np.uint8)
    Dcm_im8 = cv2.applyColorMap(Dcm_im8,cv2.COLORMAP_BONE)
    showCrosshair = False
    r = cv2.selectROI("Seleccione el ROI a analizar", Dcm_im8,showCrosshair)
    cv2.destroyAllWindows()
    # r = top x, top y, bottom x, bottom y
    return r

In [6]:
def demo_crop_geometry(pcd):
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry and to save it")
    print("5) Press 'F' to switch to freeview mode")
    pcd = pcd
    o3d.visualization.draw_geometries_with_editing([pcd])

In [7]:
image_list, image_slices, pmesh, coords, img_qtt, countmesh, result, dcm_all, dcm_im_all = [], [], [], [], 0, 0, 0, [], []
# Carga las imágenes, genera lista de slice y genera objetos
path = tk.Tk()
path.withdraw()
file_name = filedialog.askopenfilenames()

In [8]:
for i in file_name:
    Dcm = pydicom.dcmread(i)
    # Filtrado de imágenes de magnitud
    if str(Dcm[0x2005, 0x116e].value) == 'PCA': 
        image_list.append('imagen' + str(img_qtt)) # Lista de imágenes
        image_slices.append(Dcm[0x2001,0x100a].value)
        dir = 0
        # Filtrado según componentes
        if Dcm[0x2001, 0x101a].value[0]>50 and Dcm[0x2001, 0x101a].value[1]>50 and Dcm[0x2001, 0x101a].value[2]>50: # Imagen neta
            dir = 1
        elif Dcm[0x2001, 0x101a].value[0]>50 and Dcm[0x2001, 0x101a].value[1]==0 and Dcm[0x2001, 0x101a].value[2]==0: # RL # X
            dir = 2
        elif Dcm[0x2001, 0x101a].value[0]==0 and Dcm[0x2001, 0x101a].value[1]>50 and Dcm[0x2001, 0x101a].value[2]==0: # AP # Y
            dir = 3
        elif Dcm[0x2001, 0x101a].value[0]==0 and Dcm[0x2001, 0x101a].value[1]==0 and Dcm[0x2001, 0x101a].value[2]>50: # FH # Z
            dir = 4
        image_list[img_qtt] = imagen (Dcm[0x2001,0x100a].value, Dcm[0x0018,0x1060].value, np.copy(Dcm.pixel_array), Dcm, dir) # Lista de objetos tipo image
        # dcm_all.append(Dcm)
        dcm_im_all.append(Dcm.pixel_array)
        img_qtt = img_qtt + 1

In [9]:
# Ordena los objetos y calcula variables
sorted_image_list  = sorted(image_list, key = operator.attrgetter('dir', 'slice', 'time')) 
imgslctr = Counter(image_slices) # Número de slice : Cantidad de imágenes
slice_qtt = max(imgslctr.keys()) # Cantidad de slices
sl_n_imgs = int(max(imgslctr.values())/4) # Cantidad de imágenes por Slice
img_by_type = int(img_qtt/4)
height, width = (image_list[0].img).shape # Tamaño de las imágenes

In [ ]:
print(slice_qtt)

In [10]:
# Matrices a trabajar 
maskmatrix = np.zeros((slice_qtt,height,width), np.uint16)    # Máscara de todos los slice  
maskimage = np.zeros((height,width), np.uint16)               # Máscara unitaria
maskimageROI = np.zeros((height,width), np.uint64)            # Máscara unitaria para ROI
maskimgmatrix = np.zeros((slice_qtt,height,width), np.uint16) # Máscara pos-ROI de todos los slice
reconstucted_matrix = np.zeros((height,width), np.uint16)

In [11]:
# Genera el Threshold y plasma en un arreglo toda la info (en ese arreglo se va a realizar el ROI)
# Verifica que exista info en el slice anterior para eliminar vasos o info no deseada
for q in range(slice_qtt): 
    img_org = sorted_image_list[q * sl_n_imgs].img 
    prev_img_org = sorted_image_list[(q-1) * sl_n_imgs].img
    for j in range(height): 
        for k in range(width):
            if img_org[j,k] <= 80: # 500 para FFE
                maskimage[j,k] = 0
                maskimageROI[j,k] = maskimageROI[j,k] + 0
            else:
                if q == 0:
                    maskimage[j,k]= 255
                    maskimageROI[j,k] = maskimageROI[j,k] + 128
                else:
                    # Comparacion con el slice anterior (para todos menos para q=0)
                    for o in range(3):
                        for p in range(3):
                            if prev_img_org[j+o-1, k+p-1] == 255:
                                result = result + 1
                            else:
                                result = result
                    if result>0:
                        maskimage[j+o-1, k+p-1] = 255
                        maskimageROI[j+o-1, k+p-1] = maskimageROI[j+o-1, k+p-1] + 128
                    else:
                        maskimage[j+o-1, k+p-1] = 0
                        maskimageROI[j+o-1, k+p-1] = maskimageROI[j+o-1, k+p-1]
    
# Se realiza el ROI y se elimina la info fuera de él esto es para el slice maximo (me quedaba fuera de rango)
    if q == (slice_qtt-1):
        utROI = position_ROI(maskimageROI)
        for l in range(width):
            for m in range(height):
                if m>float(utROI[0]) and m<float(utROI[0]+utROI[2]) and l>float(utROI[1]) and l<float(utROI[1]+utROI[3]):
                    maskimage[l,m]=maskimage[l,m]
                else:
                    maskimage[l,m] = 0 
    maskimgmatrix[q] = maskimage

In [ ]:
# Matrices a trabajar 
maskmatrixvmtk = np.zeros((slice_qtt,height,width), np.uint16)    # Máscara de todos los slice  
maskimagevmtk = np.zeros((height,width), np.uint16)               # Máscara unitaria
maskimageROIvmtk = np.zeros((height,width), np.uint64)            # Máscara unitaria para ROI
maskimgmatrixvmtk = np.zeros((slice_qtt,height,width), np.uint16) # Máscara pos-ROI de todos los slice
reconstucted_matrixvmtk = np.zeros((height,width), np.uint16)

In [ ]:
# AGREGADO PARA SACAR UNA SOLA FASE 21/4//22
# Genera el Threshold y plasma en un arreglo toda la info (en ese arreglo se va a realizar el ROI)
# Verifica que exista info en el slice anterior para eliminar vasos o info no deseada
for q in range(slice_qtt): 
#     img_org = sorted_image_list[q * sl_n_imgs].img 
    
# Se realiza el ROI y se elimina la info fuera de él esto es para el slice maximo (me quedaba fuera de rango)
    if q == int(slice_qtt-1):
        utROIvmtk = position_ROI(maskimageROI)
        for l in range(width):
            for m in range(height):
                if m>float(utROIvmtk[0]) and m<float(utROIvmtk[0]+utROIvmtk[2]) and l>float(utROIvmtk[1]) and l<float(utROIvmtk[1]+utROIvmtk[3]):
                    maskimagevmtk[l,m]=sorted_image_list[q* sl_n_imgs].img[l,m]
                else:
                    maskimagevmtk[l,m] = 0 
    maskimgmatrixvmtk[q] = maskimagevmtk

In [ ]:
for q in range(slice_qtt):
    for l in range(width):
            for m in range(height):
                if m>float(utROIvmtk[0]) and m<float(utROIvmtk[0]+utROIvmtk[2]) and l>float(utROIvmtk[1]) and l<float(utROIvmtk[1]+utROIvmtk[3]):
                    maskimgmatrixvmtk[q,l,m]=sorted_image_list[q*sl_n_imgs].img[l,m]
                else:
                    maskimgmatrixvmtk[q,l,m] = 0

In [ ]:
# Se genera la nueva serie dicom
# print(sorted_image_list[1].info)

for i in range(slice_qtt):
    name = str(i)
    vAux1=sorted_image_list[i].info #Se busca coordenada de referencia     
    path_full=r"C:\Users\mpaul\Desktop\FUA-N19\SEC_IMGS"
    map1 = maskimgmatrixvmtk[i]
    prueba = addDCM(DCMorigen= vAux1, PathFull= path_full, ImgName= name, sizex = width, sizey = height, mapa = map1)

In [12]:
# hago lo mismo que hacia en las lineas anteriores pero para los demas slices

for q in range(slice_qtt):
    for l in range(width):
            for m in range(height):
                if m>float(utROI[0]) and m<float(utROI[0]+utROI[2]) and l>float(utROI[1]) and l<float(utROI[1]+utROI[3]):
                    maskimgmatrix[q,l,m]=maskimgmatrix[q,l,m]
                else:
                    maskimgmatrix[q,l,m] = 0

In [13]:
# Se realiza la deteccion de bordes con sobel
for q in range(slice_qtt):
    maskimageinv = cv2.flip(maskimgmatrix[q],-1)
    sobelx = cv2.Sobel(maskimgmatrix[q],-1,1,0,ksize=1) # Image , DDepth = -1 (uint8) would be the result
    sobely = cv2.Sobel(maskimgmatrix[q],-1,0,1,ksize=1) # 1 , 0  for dx   and  0 , 1 for dy
    sobeldir = sobelx + sobely
    sobelxinv = cv2.Sobel(maskimageinv,-1,1,0,ksize=1) 
    sobelyinv = cv2.Sobel(maskimageinv,-1,0,1,ksize=1)
    sobelinv = sobelxinv + sobelyinv 
    reconstucted_matrix = cv2.flip(sobelinv,-1)
    sobel = sobeldir +  reconstucted_matrix
    maskmatrix[q] = sobel

    # Seleccion de los valores internos a los bordes 
    for i in range(height):
        for j in range (width):
            if (j > 1) and (i > 1): # FIJARME DE OPTIMIZARLO
                if (maskmatrix[q][i][j] == 0) and (maskmatrix[q][i-1][j] != 0) and (maskmatrix[q][i][j-1] != 0):
                    maskmatrix[q][i][j] = 200 # 200 por poner un valor que sea exclusivo de los pixeles internos

In [ ]:
# Matrices a trabajar imagenes filtradas
secondarymatrix = np.zeros((slice_qtt,height,width), np.uint16)    # Arreglo de todos los slices  
secondaryimage = np.zeros((height,width), np.uint16)               # Imagen unitaria

In [ ]:
# Se obtiene la imagen con la segmentacion limpia
imclean, secondary_slices, secondary_counter = [], [], 0

for k in range (slice_qtt):
    img_org = sorted_image_list[k * sl_n_imgs].img
    for l in range (height):
        for m in range (width):
            if maskmatrix[k][l][m] > 0:
                # pixclean = img_org[j, k]
                secondaryimage[l,m] = img_org[l, m]
                # imclean.append(pixclean)

            else:
                # pixclean = 0
                # imclean.append(pixclean)
                secondaryimage[l,m] = 0

    secondarymatrix[k] = secondaryimage
    
    secondary_counter += 1

In [ ]:
plt.imshow(secondarymatrix[20], cmap = plt.cm.bone)
plt.show()

In [ ]:
# Se genera la nueva serie dicom
# print(sorted_image_list[1].info)

for i in range(slice_qtt):
    name = str(i)
    vAux1=sorted_image_list[i].info #Se busca coordenada de referencia     
    path_full=r"C:\Users\mpaul\Desktop\FUA-N19\SEC_IMGS"
    map1 = secondarymatrix[i]
    prueba = addDCM(DCMorigen= vAux1, PathFull= path_full, ImgName= name, sizex = width, sizey = height, mapa = map1)

In [14]:
# Se obtienen las coordenadas limpias de cada uno de los puntos de valor alto post filtro sobel
slice_points, slice_cells, countmesh, nodes = [], [], 0, []
print("antes del loop", countmesh)
for q in range (slice_qtt):
    for j in range(height):
        for k in range(width):
            if k > float(utROI[1]) and k < float(utROI[1] + utROI[3]) and j > float(utROI[0]) and j < float(utROI[0] + utROI[2]): # Ajustar al Roi
                if maskmatrix[q][j][k] > 0:
                    countmesh += 1
                    pclean = [j, k, q]
                    pmesh.append(pclean) # Podemos forzar etiquetas o podemos obtener etiquetas por defecto
                    coords.extend(pclean)
                    slice_points.append(countmesh)

                else:
                    slice_points.append(-1)

    for i in range(len(slice_points)):
        if slice_points[i] != -1:
            nodes.append(slice_points[i])

    # print(nodes[0])
    #slice_cells.append(countmesh//4)

antes del loop 0


In [2]:
# print(countmesh)
print(pmesh)

NameError: name 'pmesh' is not defined

In [15]:
# Generación de arreglo vectorial para cada punto 
vector_array = np.ndarray((countmesh,3)) # Definir point_qtt
counter = 0
for q in range(slice_qtt):
    for j in range(height): 
            for k in range(width): 
                if k > float(utROI[1]) and k < float(utROI[1] + utROI[3]) and j > float(utROI[0]) and j < float(utROI[0] + utROI[2]): # Ajustar al Roi
                        if maskmatrix[q][j][k] > 0: 
                            vector_x = sorted_image_list[((q*sl_n_imgs)+img_by_type*1)].img[j,k]
                            vector_y = sorted_image_list[((q*sl_n_imgs)+img_by_type*2)].img[j,k]
                            vector_z = sorted_image_list[((q*sl_n_imgs)+img_by_type*3)].img[j,k]
                            vector_array[counter] = [vector_x, vector_y , vector_z]
                            # coords.extend([vector_x, vector_y , vector_z])
                            counter += 1
    nodes.append(counter)
    # print(nodes[0])

In [ ]:
print(countmesh)

In [17]:
# ARMO LOS VECTORES EN PYVISTA

# scalar_array = [10]*countmesh
points = np.asarray(pmesh)
point_cloud = pv.PolyData(points)

'''raul = point_cloud.reconstruct_surface(nbr_sz = 10)

pl = pv.Plotter(shape=(1,2))
pl.add_mesh(point_cloud)
pl.add_title('Point Cloud of 3D Surface')
pl.subplot(0,1)
pl.add_mesh(raul, color=True, show_edges=True)
pl.add_title('Reconstructed Surface')
pl.show()'''

point_cloud['vectors'] = vector_array

# MESH
mesh1 = point_cloud.delaunay_3d(alpha=1.4) 
# mesh = point_cloud.mesh()

plotter = pv.Plotter()
# actor = plotter.add_mesh(mesh, color='red')
# plotter.show()

# plotter.export_obj('my_mesh')

'''COMENTADO EL 11/4'''
# glyphs = mesh.glyph(orient="vectors", scale=False, factor=0.9) #A LO MEJOR NOS TOQUE CAMBIAR EL FACTOR
actor = plotter.add_mesh(mesh1, color='red')
plotter.show()

# grid = glyphs.cast_to_unstructured_grid()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
gmsh.initialize()

N = len(nodes)
for i in range(N*3):
    coords[i] = coords[i]/N

# coords = []  # x, y, z coordinates of all the nodes
# nodes = []  # tags of corresponding nodes

vol = gmsh.model.addDiscreteEntity(3)
gmsh.model.mesh.addNodes(3, vol, nodes, coords)



In [ ]:
# https://gitlab.onelab.info/gmsh/gmsh/-/issues/1000


point_sets = [pmesh]

gmsh.initialize()

# loop over point sets
for i, set in enumerate(point_sets):
    # loop over point array rows
    for [x, y, z] in set:
        gmsh.model.geo.addPoint(x, y, z)

    # only synchronize after adding all the points to the model
    gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(1)
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)
gmsh.write("model" + ".msh")

# start with empty model again
# gmsh.clear()
gmsh.fltk.run()

gmsh.finalize()

In [ ]:
# # Llamese celda a un grupo de puntos con relaciones espaciales y geometricas
# import vtk

# # Prueba
# cell_size = int(4) # Cantidad de puntos por celda
# cell_qtt = 8 # Cantidad total de celdas

# points = points # Arreglo con las coordenadas de los puntos 

# cells = []

# for k in range((cell_size+1)*slice_qtt):
#     if multiplo5(k)==True:
#         cells.append(cell_size)
#     elif multiplo5(k+1)==True:
#         cells.append(cell_size)



In [ ]:
'''for i in range(cell_qtt):
    for k in range(slice_points):
        aux = [cell_size, a, b, c, d] # Indice del punto que necesito # Esto falta
        print (aux) 
    if i == 0:
        cells = aux
    else:
        cells = np.append(cells, aux)

# Definición del tipo de celda
celltypes = np.empty(cell_qtt, dtype=np.uint8)
celltypes[:] = vtk.VTK_TETRA

# Punto inicial de cada celda
offset = np.empty(cell_qtt, dtype=np.uint8)
index = 0
for i in range(0, (cell_size*cell_qtt) + 1, cell_size + 1):
    offset[index] = i
    index += 1

# Generación del Unstructured grid
meh = pv.UnstructuredGrid(offset, cells, celltypes, points)

# Visualizar
meh.plot(show_edges=True)'''

In [18]:
pl = pv.Plotter()
pl.add_mesh(mesh1, color="grey", ambient=0.6, opacity=0.1, show_edges=True)
# pl.add_mesh(grid,  lighting=False, scalar_bar_args={'title': "Vector Magnitude"})
pl.add_points(mesh1.points, color='red')
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)